<a href="https://colab.research.google.com/github/mpartain1/sales-predictions/blob/main/sales_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd

In [28]:
df = pd.read_csv('/content/sales_predictions.csv')

In [49]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300000,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920000,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500000,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200000,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930000,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395000,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650000,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,12.857645,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200000,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200000,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


In [80]:
df.shape
#1) 8523 rows, 12 columns

(8523, 12)

In [81]:
df.info()
#2) See below for data types of each variable under Dtype

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                8523 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 1.1+ MB


In [82]:
#3) No duplicates, but wanted to show the code nonetheless
print(df.duplicated().sum())
df = df.drop_duplicates()
print(df.duplicated().sum())

0
0


In [83]:
#4) 1463 missing values for Item_Weight, 2410 missing values for Outlet_Size
df.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [84]:
#5)
grouped = df.groupby('Item_Type')
grouped_means = grouped['Item_Weight'].mean()
print(grouped_means)
#Notice that all the group means hover around 12
mean_weight = df['Item_Weight'].mean()
print(mean_weight)
df['Item_Weight'].fillna(mean_weight, inplace = True)
#For Item_Weight, I addressed the missing values by taking the mean
#weight of every item since this method was simple and made sense.

df['Item_Outlet_Sales'].describe().round(2)
for i in range (df.shape[0]):
  if (df['Item_Outlet_Sales'][i] > 3101):
    df['Outlet_Size'].fillna('High', inplace = True)
  elif ((df['Item_Outlet_Sales'][i] > 834) & (df['Item_Outlet_Sales'][i] < 3101)):
    df['Outlet_Size'].fillna('Medium', inplace = True)
  else:
    df['Outlet_Size'].fillna('Small', inplace = True)
#For Outlet_Size, I filled in missing values according to the
#pattern of Item_Outlet_Sales since I would imagine there is a 
#positive correlation between the two.




Item_Type
Baking Goods             12.377448
Breads                   11.629818
Breakfast                12.785278
Canned                   12.399254
Dairy                    13.329387
Frozen Foods             12.865543
Fruits and Vegetables    13.161297
Hard Drinks              11.611435
Health and Hygiene       13.093044
Household                13.297274
Meat                     12.825689
Others                   13.664761
Seafood                  12.614756
Snack Foods              12.964871
Soft Drinks              12.008636
Starchy Foods            13.589410
Name: Item_Weight, dtype: float64
12.857645184136409


In [85]:
#6)
df.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [90]:
#Weights stats
df['Item_Weight'].describe()
#MRP stats
df['Item_MRP'].describe()
#Visibility stats
df['Item_Visibility'].describe()
#Sales stats
df['Item_Outlet_Sales'].describe()

count     8523.000000
mean      2181.288914
std       1706.499616
min         33.290000
25%        834.247400
50%       1794.331000
75%       3101.296400
max      13086.964800
Name: Item_Outlet_Sales, dtype: float64